# On-balance volume

* It measures the **buy** and **sell** pressures
* It uses the **volume**
* Can be used to
  * Measure the **strength** of a trend
  * **Identify** or **confirm** possible reversions
  * **Show** the beginning of a **new trend**

**Attention:** these sites ([[1]](https://school.stockcharts.com/doku.php?id=technical_indicators:on_balance_volume_obv), [[2]](https://www.investopedia.com/terms/o/onbalancevolume.asp)) define the OBV as:

```pseudocode
if close_price > close_price_prev:
	OBV = OBV_prev + volume_current
	
if close_price < close_price_prev:
	OBV = OBV_prev - volume_current

if close_price == close_price_prev: //else
	OBV = OBV_prev
```

My book defines it as

```pseudocode
if close_price > open_price:
	OBV = OBV_prev + volume_current
	
if close_price < open_price:
	OBV = OBV_prev - volume_current

if close_price == close_price_prev: //else
	close_price = open_price //?
```



## OBV Analisys

* **OBV Growth**
  * There are more **buyers** then sellers
  * The prices will **raise**
* **OBV degrowth**
  * There are more **sellers** then buyers
  * The prices will **decrease**



## OBV Patterns

* **Divergence**
  * When the OBV trend "disagrees" with the price trend - when they're going in opposite directions
  * **Strong pattern**: *OBV down Price up*
  * Divergences indicates possible opportunities for buying or selling depending on the **size** of the divergence
* **Confirmation**
  * Can confirm new trends by applying support and resistance on the OBV chart

In [1]:
import sys
import os

import chart_studio.plotly as plty
import plotly.graph_objs as gobjs

import plotly

sys.path.insert(0, os.path.abspath('../py'))
from Secrets import ReadSecrets


account = ReadSecrets()\
        .set_secrets_path('../secrets.json')\
        .set_default_accessors()\
        .access('plotly')['main_account']

plty.sign_in(account['username'], account['apiKey'])


from datetime import datetime as calendar
import numpy as np

from pandas_datareader import data as pdr
import pandas as pd
import yfinance as yf
yf.pdr_override()

from DataHelper import DataHelper

from plotly import tools

/home/andre/Desktop/pythonTradingAlgos/tradingalgos/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [2]:
data, close, extractors = DataHelper.get_history_formatted(['AAPL', 'MSFT', '^GSPC'], calendar(2016, 1 ,1), calendar(2020, 2, 1))
msft = extractors['MSFT']()
aapl = extractors['AAPL']()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
obv = pd.DataFrame(index=aapl.index)

# get the percentual changes between day[i] and day[i-1]
obv_changes = aapl.Close - aapl.Open

obv['open'] = aapl.Open
obv['close'] = aapl.Close
obv['volume'] = np.where(obv_changes > 0, aapl.Volume , aapl.Volume * (-1))

# get days with positive profit (gain)
obv['volume_sum'] = obv.volume.cumsum()

In [6]:
trace_obv = gobjs.Scatter(
                    x=obv.index,
                    y=obv.volume_sum,
                    xaxis='x2',
                    yaxis='y2')

trace_candles = gobjs.Candlestick(x=aapl.index,
                    open=aapl.Open,
                    high=aapl.High,
                    low=aapl.Low,
                    close=aapl.Close)

data = [trace_obv, trace_candles]

layout = gobjs.Layout(
    xaxis=dict(
        domain=[0, 1],
        rangeslider={"visible": False},
    ),
    yaxis=dict(
        domain=[0.75, 1],        
    ),
    xaxis2=dict(
        domain=[0, 1],
        anchor='y2'
        
    ),
    yaxis2=dict(
        domain=[0, 0.45],
        anchor='x2'
        
    )
)
fig = gobjs.Figure(data=data, layout=layout)

plty.iplot(fig, filename='obv-plot')



In [38]:
obv = pd.DataFrame(index=aapl.index, data={
    'Close': aapl.Close,
    'PrevClose': aapl.Close.shift(1),
    'Volume': aapl.Volume,
    'PrevVolume': aapl.Volume.shift(1),
})


obv['ObvGreater'] = np.where(obv.Close > obv.PrevClose, obv.Volume, 0)
obv['ObvSmaller'] = np.where(obv.Close < obv.PrevClose, -obv.Volume, 0)
obv['ObvVolume'] = obv.ObvGreater + obv.ObvSmaller
obv['ObvVolumeCumulative'] = obv.ObvVolume.cumsum()


In [40]:
trace_obv = gobjs.Scatter(
                    x=obv.index,
                    y=obv.ObvVolumeCumulative,
                    xaxis='x2',
                    yaxis='y2')

trace_candles = gobjs.Candlestick(x=aapl.index,
                    open=aapl.Open,
                    high=aapl.High,
                    low=aapl.Low,
                    close=aapl.Close)

data = [trace_obv, trace_candles]

layout = gobjs.Layout(
    xaxis=dict(
        domain=[0, 1],
        rangeslider={"visible": False},
    ),
    yaxis=dict(
        domain=[0.75, 1],        
    ),
    xaxis2=dict(
        domain=[0, 1],
        anchor='y2'
        
    ),
    yaxis2=dict(
        domain=[0, 0.45],
        anchor='x2'
        
    )
)
fig = gobjs.Figure(data=data, layout=layout)

plty.iplot(fig, filename='obv-plot')